In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import datasets
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import random
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,accuracy_score
from sklearn.utils import resample

%matplotlib inline

In [2]:
df = pd.read_csv('KaggleV2-May-2016.csv')

### whole set

In [3]:
X_train, X_test  = train_test_split(df,test_size=0.33,random_state=42)
print(f'Train set with attributes has {X_train.shape[0]} rows and Test set has {X_test.shape[0]} rows')

Train set with attributes has 74053 rows and Test set has 36474 rows


### target set balance

In [4]:
percent_YES_train = X_train['No-show'].value_counts()[1]/X_train['No-show'].value_counts().sum()
percent_YES_test = X_test['No-show'].value_counts()[1]/X_test['No-show'].value_counts().sum()
#print('We have ca. 20% YES and 80% NO in both sets')

### gander balanced sets

In [6]:
X_train.Gender.value_counts()
percent_Woman_train = round(X_train.Gender.value_counts()/X_train.Gender.value_counts().sum()*100)
percent_Woman_test = round(X_test.Gender.value_counts()/X_test.Gender.value_counts().sum()*100)

print('Our train and test sets with attributes have 65% woman in both of them')
percent_Woman_train 

Our train and test sets with attributes have 65% woman in both of them


F    65.0
M    35.0
Name: Gender, dtype: float64

## stratified suffle split 

In [9]:
df_without_target = df.drop('No-show',axis=1)
target = df['No-show']
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)

for train_index,test_index in split.split(df_without_target,target):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_s, X_test_s = df.iloc[train_index], df.iloc[test_index]
    y_train_s, y_test_s = target[train_index], target[test_index]

## unique patients in train and test

In [10]:
patient_list = df.PatientId.unique()
print(patient_list)
random.seed(42)
np.random.shuffle(patient_list)

print(patient_list)

size_of_test = round(len(patient_list)*0.25)
size_of_train = round(len(patient_list)*0.75)

patient_train = patient_list[15575:]
patient_test =  patient_list[:15575]

train = df[df['PatientId'].isin(patient_train)]
test = df[df['PatientId'].isin(patient_test)]


[2.98724998e+13 5.58997777e+14 4.26296230e+12 ... 7.26331493e+13
 9.96997666e+14 1.55766317e+13]
[2.42447297e+13 9.76948596e+11 2.66868977e+12 ... 4.26275768e+13
 7.72542249e+12 5.97675484e+14]


### train-test split comparision

In [15]:
sampling_bias_comparision = pd.DataFrame()
whole_set_gender = df.Gender.value_counts()/len(df)
sampling_bias_comparision['overall_gender'] = whole_set_gender

sampling_bias_comparision_target = pd.DataFrame()
sampling_bias_comparision_target['target_oberall'] = target.value_counts()/len(target)

sampling_bias_comparision['train_test_split_gender_TRAIN'] = X_train.Gender.value_counts()/len(X_train)
sampling_bias_comparision['train_test_split_gender_TEST'] = X_test.Gender.value_counts()/len(X_test)

sampling_bias_comparision_target['train_test_split_target_TRAIN'] = X_train['No-show'].value_counts()/len(X_train['No-show'])
sampling_bias_comparision_target['train_test_split_target_TEST'] = X_test['No-show'].value_counts()/len(X_test['No-show'])



sampling_bias_comparision['StratifiedShufflesplit_TRAIN'] = X_train_s.Gender.value_counts()/len(X_train_s)
sampling_bias_comparision['StratifiedShufflesplit_TEST'] = X_test_s.Gender.value_counts()/len(X_test_s)

sampling_bias_comparision_target['StratifiedShufflesplit_target_TRAIN'] = y_train_s.value_counts()/len(y_train_s)
sampling_bias_comparision_target['StratifiedShufflesplit-taget_TEST'] = y_test_s.value_counts()/len(y_test_s)

sampling_bias_comparision_target['UniquePatients_TRAIN'] = train['No-show'].value_counts()/len(train)
sampling_bias_comparision_target['UniquePatients_TEST'] = test['No-show'].value_counts()/len(test)

sampling_bias_comparision_target

,target_oberall,train_test_split_target_TRAIN,train_test_split_target_TEST,StratifiedShufflesplit_target_TRAIN,StratifiedShufflesplit-taget_TEST,UniquePatients_TRAIN,UniquePatients_TEST
No,0.798067,0.797172,0.799885,0.798068,0.798064,0.797856,0.798701
Yes,0.201933,0.202828,0.200115,0.201932,0.201936,0.202144,0.201299
